In [3]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# second convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Step 3 - Flattening
classifier.add(Flatten())

In [6]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [7]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 800,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 200)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
800/800 [==============================] - 306s 382ms/step - loss: 0.6471 - acc: 0.6123 - val_loss: 0.6089 - val_acc: 0.6806
Epoch 2/25
800/800 [==============================] - 279s 349ms/step - loss: 0.5177 - acc: 0.7410 - val_loss: 0.4968 - val_acc: 0.7604
Epoch 3/25
800/800 [==============================] - 278s 348ms/step - loss: 0.4487 - acc: 0.7869 - val_loss: 0.4553 - val_acc: 0.7868
Epoch 4/25
800/800 [==============================] - 278s 348ms/step - loss: 0.4040 - acc: 0.8149 - val_loss: 0.4565 - val_acc: 0.8012
Epoch 5/25
800/800 [==============================] - 279s 348ms/step - loss: 0.3663 - acc: 0.8332 - val_loss: 0.4198 - val_acc: 0.8068
Epoch 6/25
800/800 [==============================] - 279s 348ms/step - loss: 0.3270 - acc: 0.8559 - val_loss: 0.4436 - val_acc: 0.8160
Epoch 7/25
800/800 [==============================] - 279s 349ms/step - loss: 0.2891 - acc: 0.8762 - val_loss: 0.4603 - val_acc: 0.8169
